In [1]:
import csv
from bs4 import BeautifulSoup

In [2]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
driver = webdriver.Chrome(ChromeDriverManager().install())

In [4]:
url = 'https://www.amazon.com'
driver.get(url)

In [5]:
def get_url(search_term):
    template = 'https://www.amazon.com/s?i=pets-intl-ship&bbn=16225013011&rh=n%3A2975312011%2Cn%3A2975359011&dc&ds=v1%3ASxyY%2BRu8LEGJNuVH1uyz6ZIkIB6YBwbExS599BB82OA&qid=1666671521&rnid=2975312011&ref=sr_nr_n_1'
    search_term = search_term.replace('', '+')
    return template.format(search_term)

In [6]:
url = get_url('ultrawide monitor')
print(url)

https://www.amazon.com/s?i=pets-intl-ship&bbn=16225013011&rh=n%3A2975312011%2Cn%3A2975359011&dc&ds=v1%3ASxyY%2BRu8LEGJNuVH1uyz6ZIkIB6YBwbExS599BB82OA&qid=1666671521&rnid=2975312011&ref=sr_nr_n_1


In [7]:
driver.get(url)

# Extract the collection ##

In [8]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [9]:
results = soup.find_all('div', {'data-component-type' : 's-search-result'})

In [10]:
len(results)

26

## prototype the record ##

In [11]:
item = results[0]

In [12]:
atag = item.h2.a

In [13]:
description = atag.text.strip()

In [14]:
url = 'https://amazon.com' + atag.get('href')

In [15]:
price_parent = item.find('span', 'a-price')

In [16]:
price = price_parent.find('span', 'a-offscreen').text

In [17]:
rating = item.i.text

In [18]:
review_count = item.find('span', {'class' : 'a-size-base'}).text

## Generalize the pattern ##

In [19]:
def extract_record(item):
    """Extract and return data from a single record"""
    #description and url
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://amazon.com' + atag.get('href')
    
    #price
    #price_parent = item.find('span', 'a-price')
    #price = price_parent.find('span', 'a-offscreen').text
    
    #rank and rating
    rating = item.i.text
    review_count = item.find('span', {'class' : 'a-size-base'}).text
    
    result = (description, rating, review_count, url)
    
    return result

In [20]:
records = []
results = soup.find_all('div', {'data-component-type' : 's-search-result'})

for item in results:
    records.append(extract_record(item))

## Error handling ##

In [21]:
def extract_record(item):
    """Extract and return data from a single record"""
    #description and url
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://amazon.com' + atag.get('href')
    
    #try :
        #price
        #price_parent = item.find('span', 'a-price')
        #price = price_parent.find('span', 'a-offscreen').text
    #except AttributeError:
        #return
    
    try:    
        #rank and rating
        rating = item.i.text
        review_count = item.find('span', {'class' : 'a-size-base'}).text
    except AttributeError:
        rating = ''
        review_count = ''
    
    result = (description, rating, review_count, url)
    
    return result

In [22]:
records = []
results = soup.find_all('div', {'data-component-type' : 's-search-result'})

for item in results:
    record = extract_record(item)
    if record:
        records.append(record)

In [23]:
records[0]

('HOLI Freeze Dried Salmon Dog Food Topper - with Omega 3 & 6 - Single Ingredient, Human-Grade Wild Caught Salmon - Freeze Dried Dog Food Flavor Enhancer for Picky Dogs – 100% All Natural',
 '4.3 out of 5 stars',
 '206',
 'https://amazon.com/sspa/click?ie=UTF8&spc=MToyMzAxNjk4MjcxODUxNDI5OjE2NjY3MTA1Nzk6c3BfYXRmX2Jyb3dzZToyMDAwOTIwODQ5MjE5OTg6OjA6Og&sp_csd=d2lkZ2V0TmFtZT1zcF9hdGZfYnJvd3Nl&url=%2FHOLI-Caught-Salmon-Ingredient-Protein%2Fdp%2FB08PYM3S4S%2Fref%3Dsr_1_1_sspa%3Fqid%3D1666710579%26rnid%3D2975312011%26s%3Dpet-supplies%26sr%3D1-1-spons%26psc%3D1')

In [24]:
for row in records:
    print(row[1])

4.3 out of 5 stars
4.7 out of 5 stars
4.7 out of 5 stars
4.7 out of 5 stars
4.8 out of 5 stars
4.7 out of 5 stars
4.7 out of 5 stars
4.7 out of 5 stars
4.7 out of 5 stars
4.7 out of 5 stars
4.7 out of 5 stars
4.7 out of 5 stars
4.0 out of 5 stars
4.6 out of 5 stars
4.7 out of 5 stars
4.7 out of 5 stars
4.6 out of 5 stars
4.8 out of 5 stars
4.7 out of 5 stars
4.7 out of 5 stars
4.7 out of 5 stars
4.7 out of 5 stars
4.6 out of 5 stars
4.7 out of 5 stars
4.7 out of 5 stars
4.6 out of 5 stars


## getting the next page ##

In [25]:
def get_url(search_term):
    template = 'https://www.amazon.com/s?i=pets-intl-ship&bbn=16225013011&rh=n%3A2975312011%2Cn%3A2975359011&dc&ds=v1%3ASxyY%2BRu8LEGJNuVH1uyz6ZIkIB6YBwbExS599BB82OA&qid=1666671521&rnid=2975312011&ref=sr_nr_n_1'
    search_term = search_term.replace('', '+')
    
    #add term query to url
    url = template.format(search_term)
    
    #add page query placeholder
    url += '&page{}'
    
    return url

## putting it all together ##

In [26]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

def get_url(search_term):
    template = 'https://www.amazon.com/s?i=pets-intl-ship&bbn=16225013011&rh=n%3A2975312011%2Cn%3A2975359011&dc&={}qid=1666709157&rnid=2975312011&ref=sr_pg_1'
    search_term = search_term.replace(' ', '+')
    
    #add term query to url
    url = template.format(search_term)
    
    #add page query placeholder
    url += '&page{}'
    
    return url

def extract_record(item):
    """Extract and return data from a single record"""
    #description and url
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://amazon.com' + atag.get('href')
    
    
    try:    
        #rank and rating
        rating = item.i.text
        review_count = item.find('span', {'class' : 'a-size-base'}).text
    except AttributeError:
        rating = ''
        review_count = ''
    
    result = (description, rating, review_count, url)
    
    return result

def main(search_term):
    """Run main program routine"""
    #startup the webdriver
    driver = webdriver.Chrome(ChromeDriverManager().install())
    
    record = []
    url = get_url(search_term)
    
    for page in range(1, 25):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div', {'data-component-type' : 's-search-result'})
        
        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)
                
    driver.close()
    
    #save sata to csv file
    with open('results.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Description', 'Rating', 'ReviewCount', 'Url'])
        writer.writerows(records)

In [27]:
main('ultrawide monitor')